# <p style="color:dodgerblue">Text Generation in Bedrock</p>
<hr style="border:1px dotted; color:floralwhite">

# Requirements for this lab
*See <span style="color:gold">Appendix</span> at the bottom of this lab to install requirements for this lab*

<hr style="border:1px dotted">
<hr style="border:1px dotted;color:cadetblue">

# <p style="color:cadetblue">Import libraries and set clients</p>
*If the following does not show the latest (installed or upgraded) boto3 (at least v1.29.6), restart the kernel*

In [1]:
import boto3, json

# region_name is important especially if you auth via aws configure and it is set to a region where there is no Bedrock yet
#define common vars
myRegion='us-west-2'

# get clients
bedrockRun = boto3.client(service_name='bedrock-runtime', region_name=myRegion)

boto3.__version__

'1.33.9'

<hr style="border:1px dotted;color:cadetblue">
<hr style="border:1px dotted;color:crimson">

# <p style="color:crimson">Create some methods to call</p>
1. Method for processing a prompt
2. Method to have a conversation
3. <b>Don't forget if you modify this method, make sure you execute the cell</b>

### <p style="color:crimson">This method processes a single prompt with the chosen foundation model</p>

In [2]:
# processes a single prompt
def processPrompt(m, prompt, p, t, k, n):
    # prep some vars
    body=''
    outputResponse='Model: {}\n---------------------------------------\n'.format(m)
    accept='application/json'
    contentType='application/json'
    tk=2000

    # which model are we calling
    match m:
        case 'ai21.j2-ultra-v1' | 'ai21.j2-mid-v1': 
            # AI21 Labs - Jurassic-2 Ultra and Mid 
            # https://www.ai21.com/studio
            body = json.dumps({
                "prompt": prompt,
                "temperature": t, 
                "topP": p, 
                "maxTokens": tk,
                "numResults": n,
            })
        case 'amazon.titan-text-express-v1':
            # Amazon - Titan
            # https://aws.amazon.com/bedrock/titan/
            body = json.dumps({
                "inputText": prompt,
                "textGenerationConfig": {
                    "temperature": t,
                    "topP": p,
                    "maxTokenCount": tk,
                },
            })
        case 'cohere.command-text-v14':
            # Cohere - Command
            # https://cohere.com/
            body = json.dumps({
                "prompt": prompt,
                "temperature": t,
                "p": p,
                "k": k,
                "max_tokens": tk,
                "num_generations": n,
            })
        case 'mistral.mistral-7b-instruct-v0:2':
            # Mistral - Mistral
            # https://mistral.ai/
            body = json.dumps({
                "prompt": prompt,
                "temperature": t,
                "top_p": p,
                "top_k": k,
                "max_tokens": tk,
            })
        case 'meta.llama2-13b-chat-v1':
            # Meta - Llama 2
            # https://llama.meta.com/
            body = json.dumps({
                "prompt": prompt,
                "temperature": t,
                "top_p": p,
                "max_gen_len": tk,
            })
        case 'anthropic.claude-v2:1':
            # Anthropic - Claude
            # https://www.anthropic.com/
            body=json.dumps({
                "prompt": "\n\nHuman:{}\n\nAssistant:".format(prompt),
                "temperature": t,
                "top_p": p,
                "top_k": k,
                "max_tokens_to_sample": tk,
            })  

    #invoke the model
    response = bedrockRun.invoke_model(body=body, modelId=m, accept=accept, contentType=contentType)
    responseBody = json.loads(response.get('body').read())

    # get the response
    match m:
        case 'ai21.j2-ultra-v1' | 'ai21.j2-mid-v1':            
            for response in range(n):
                outputText = responseBody.get('completions')[response].get('data').get('text')
                outputResponse += 'RESPONSE: {}\n{}\n---------------------------------------\n'.format(response+1, outputText)
        case 'amazon.titan-text-express-v1':
            outputText = responseBody['results'][0]['outputText']
            outputResponse += '1 RESPONSE Supported:\n{}\n---------------------------------------\n'.format(outputText)
        case 'cohere.command-text-v14':
            for response in range(n):
                outputText = responseBody['generations'][response]['text']
                outputResponse += 'RESPONSE: {}\n{}\n---------------------------------------\n'.format(response+1, outputText)
        case 'mistral.mistral-7b-instruct-v0:2':
            outputText = responseBody['outputs'][0]['text']
            outputResponse += '1 RESPONSE Supported:\n{}\n---------------------------------------\n'.format(outputText)
        case 'meta.llama2-13b-chat-v1':
            outputText = responseBody['generation']
            outputResponse += '1 RESPONSE Supported:\n{}\n---------------------------------------\n'.format(outputText)
        case 'anthropic.claude-v2:1':
            outputText = responseBody['completion']
            outputResponse += '1 RESPONSE Supported:\n{}\n---------------------------------------\n'.format(outputText)

    # return response
    return outputResponse

### <p style="color:crimson">This following methods create a chatbot with the chosen foundation model</p>

In [3]:
# process chatbot conversation
# langchain required as models are stateless
from langchain.llms.bedrock import Bedrock
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain import PromptTemplate

def chatBot(m, p, t): # demo_chatbot
    # prep some vars
    tk = 1000

    # following connections the FM to langchain
    # dont need a prompt yet as we haven't started the conversation
    match m:
        case "ai21.j2-ultra-v1" | "ai21.j2-mid-v1":
            # AI21 Labs - Jurassic-2 Ultra and Mid 
            # https://www.ai21.com/studio
            cl_llm = Bedrock(
                model_id=m,
                client=bedrockRun,
                model_kwargs={
                    "temperature": t, 
                    "topP": p,
                    "maxTokens": tk,
                },
            )
        case 'amazon.titan-text-express-v1':
            # AI21 Labs - Jurassic-2 Ultra and Mid 
            # https://www.ai21.com/studio
            cl_llm = Bedrock(
                model_id=m,
                client=bedrockRun,
                model_kwargs={
                    "temperature": t,
                    "topP": p,
                    "maxTokenCount": tk,
                },
            )
        case "meta.llama2-13b-chat-v1" | "meta.llama2-70b-chat-v1":
            # Meta - Llama 2
            # https://llama.meta.com/
            cl_llm = Bedrock(
                model_id=m,
                client=bedrockRun,
                model_kwargs={
                    "temperature": t,
                    "top_p": p,
                    "max_gen_len": tk,
                },
            )
        case 'anthropic.claude-v2:1':
            # Anthropic - Claude
            # https://www.anthropic.com/
            cl_llm = Bedrock(
                model_id=m,
                client=bedrockRun,
                model_kwargs={
                    "temperature": t,
                    "top_p": p,
                    "max_tokens_to_sample": tk,
                    "stop_sequences": ["\n\nHuman:"]
                },
            )

    return cl_llm

In [ ]:
# chatbot memory to remember conversations
def chatBotMemory(m, p, t): # demo_memory
    llmData=chatBot(m, p, t)
    memory = ConversationBufferMemory(llm=llmData, max_token_limit=5000)
    return memory


In [ ]:
# chatbot conversation chain
def chatBotChain(prompt, m, p, t, memory): # demo_conversation
    llmChain=chatBot(m, p, t)
    llmConversation=ConversationChain(llm=llmChain, memory=memory, verbose=True)

    response=llmConversation.predict(input=prompt)
    return response

In [ ]:
# process prompt suitable for the model
def chatBotPrompt(prompt, m, useTemplate):
    # provide a template to stop model from simulation Human replies and continuing the conversation on its own
    # you can tell the model how to answer, eg you are a primary school teacher, or answer in French, etc
    if useTemplate:
        t='{who1}[INST]You are a helpful bot which answers one question at a time.[/INST]{who2}\n{who1}{human}{who2}'
    else:
        t='{who1}{human}{who2}'

    match m:
        case "ai21.j2-ultra-v1" | "ai21.j2-mid-v1":
            # AI21 Labs - Jurassic-2 Ultra and Mid 
            # https://www.ai21.com/studio
            prompt=t.format(who1='', human=prompt, who2='')
        case 'amazon.titan-text-express-v1':
            # AI21 Labs - Jurassic-2 Ultra and Mid 
            # https://www.ai21.com/studio
            prompt=t.format(who1='User: ', human=prompt, who2='\nBot:')
        case "meta.llama2-13b-chat-v1" | "meta.llama2-70b-chat-v1":
            # Meta - Llama 2
            # https://llama.meta.com/
            prompt=t.format(who1='', human=prompt, who2='')
        case 'anthropic.claude-v2:1':
            # Anthropic - Claude
            # https://www.anthropic.com/
            prompt=t.format(who1='\n\nHuman: ', human=prompt, who2='\n\nAssistant:')

    return prompt

# <p style="color:crimson">Now we can start using our models with prompts</p>

<hr style="border:1px dotted;color:crimson">
<hr style="border:1px dotted;color:lightblue">

# <p style="color:lightblue">Invoke a model with a single prompt</p>
1. Specify properties
2. Call the method
3. Prompt engineering <a href='https://docs.aws.amazon.com/bedrock/latest/userguide/general-guidelines-for-bedrock-users.html'>help</a>

In [ ]:
# set up the prompt and params - note each model will have different ranges for its params
prompt='You are a primary school teacher teaching 10 year old children. Write a class presentation about our solar system. Answer in Thai.'
t=0.7 # temperature: use a lower value to decrease randomness in the response
p=0.5 # topP: use a lower value to ignore less probable options
k=50 # topK: number of most-likely candidates that the model considers for the next token
n=2 # numResults: how many responses do you want, not all models support multiple responses

# which model do you want to use based on the match statement below
u=7

match u:
    case -1: # NOTE NOT AVAILABLE FOR THIS LAB
        # AI21 Labs - Jurassic-2 Ultra
        # property ranges: https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-jurassic2.html
        m='ai21.j2-ultra-v1'
    case -2: # NOTE NOT AVAILABLE FOR THIS LAB
        # AI21 Labs - Jurassic-2 Mid
        # property ranges: https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-jurassic2.html
        m='ai21.j2-mid-v1'
    case 3:
        # Amazon - Titan
        # property ranges: https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-titan-text.html
        m='amazon.titan-text-express-v1'
    case -4: # NOTE NOT AVAILABLE FOR THIS LAB
        # Cohere - Command
        # property ranges: https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-cohere-command.html
        m='cohere.command-text-v14'
    case 5:
        # Mistral - Mistral
        # property ranges: https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-mistral.html
        m='mistral.mistral-7b-instruct-v0:2'
    case -6: # NOTE NOT AVAILABLE FOR THIS LAB
        # Meta - Llama 2
        # https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-meta.html
        m='meta.llama2-13b-chat-v1'
    case 7:
        # Anthropic - Claude
        # https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-text-completion.html
        m='anthropic.claude-v2:1'

o=processPrompt(m, prompt, p, t, k, n)
print(o)

<hr style="border:1px dotted;color:lightblue">
<hr style="border:1px dotted;color:deeppink">


# <p style="color:deeppink">Invoke a chatbot model with a conversational prompt</p>
1. Specify properties
2. Call the method
3. Prompt engineering <a href='https://docs.aws.amazon.com/bedrock/latest/userguide/general-guidelines-for-bedrock-users.html'>help</a>

In [ ]:
# set up the prompt and params - note each model will have different ranges for its params
prompt='hello'
t=0 # temperature: use a lower value to decrease randomness in the response
p=0.1 # topP: use a lower value to ignore less probable options
k=50 # topK: number of most-likely candidates that the model considers for the next token

# which model do you want to use based on the match statement below
u=3

match u:
    case -1: # NOTE NOT AVAILABLE FOR THIS LAB
        # AI21 Labs - Jurassic-2 Ultra
        # property ranges: https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-jurassic2.html
        m='ai21.j2-ultra-v1'
    case -2: # NOTE NOT AVAILABLE FOR THIS LAB
        # AI21 Labs - Jurassic-2 Mid
        # property ranges: https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-jurassic2.html
        m='ai21.j2-mid-v1'
    case 3:
        # Amazon - Titan
        # property ranges: https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-titan-text.html
        m='amazon.titan-text-express-v1'
    case -4: # NOTE NOT AVAILABLE FOR THIS LAB
        # Meta - Llama 2
        # https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-meta.html
        m='meta.llama2-70b-chat-v1'
    case 5:
        # Anthropic - Claude
        # https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-text-completion.html
        m='anthropic.claude-v2:1'

# format prompt suitable for conversation
prompt=chatBotPrompt(prompt, m, True)
print("YOUR PROMPT FORMATTED:\n" + prompt)
# start chatbot
memory=chatBotMemory(m, p, t)
o=chatBotChain(prompt, m, p, t, memory)
print(o)

In [ ]:
# continue the conversation
prompt=chatBotPrompt("How old is planet earth?", m, False)
o=chatBotChain(prompt, m, p, t, memory)
print(o)

In [ ]:
# continue the conversation
prompt=chatBotPrompt("How do you know that?", m, False)
o=chatBotChain(prompt, m, p, t, memory)
print(o)

<hr style="border:1px dotted;color:crimson">
<hr style="border:1px dotted;color:lightgreen">


# <p style="color:lightskyblue">Anthropic - Claude - Using the Claude Massage API - Latest Gen</p>
1. Set up Bedrock request
2. Set the controls (https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-messages.html)
3. Invoke the model with the prompt

In [ ]:
 # NOTE NOT AVAILABLE FOR THIS LAB
def claudeMessageAPI(bedrock_runtime, model_id, system_prompt, messages, max_tokens):

    body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "system": system_prompt,
            "messages": messages
        }  
    )  

    
    response = bedrock_runtime.invoke_model(body=body, modelId=model_id)
    response_body = json.loads(response.get('body').read())
   
    return response_body

# https://docs.anthropic.com/claude/reference/claude-on-amazon-bedrock
modelIdC = 'anthropic.claude-3-haiku-20240307-v1:0'

# Prompt with user turn only.
user_message =  {"role": "user", "content": "You are the anthropic.claude-3-haiku-20240307-v1 foundation model. Describe yourself in the first person using a maximum of 200 words on the following topics:  1. What you have been trained on. 2. Your purpose. 3. Your proudest achievement. 4. Provide detail of up to 5 parameters of yourself listing their numeric values if appropriate. 6. Tell us what you hope for your future."}
messages = [user_message]
systemPromptC = "Please respond in Thai."

response = claudeMessageAPI (bedrockRun, modelIdC, systemPromptC, messages, 2000)

print(json.dumps(response, indent=4))
text_value = response['content'][0]['text']
print(text_value)

<hr style="border:1px dotted;color:coral">
<hr style="border:1px dotted;color:gold">

# <p style="color:gold">Appendix - Install Requirements (macOS)</p>
# Requirements for this lab

#### <p style="color:deeppink">- If you are running VSCode on a laptop, follow all of below.<br>- If you are running Jupyter inside an AWS Account, you just need to install langchain. See 2.1 below and skip everything else.</p>

*Windows requirements will be similar, apart from Homebrew.*  
* python must be installed on your client, and be at least v3.8
  * 3.8 supports the latest release of boto3 which supports Bedrock  
* boto3 must be installed on your client, and be at least v1.33.9  
  * *Boto3 is the Amazon Web Services (AWS) Software Development Kit (SDK) for Python, which allows Python developers to write software that makes use of services like Amazon S3 and Amazon EC2.*
  * https://boto3.amazonaws.com/v1/documentation/api/latest/index.html
### <p style="color:gold">1. Homebrew</p> 
If you haven't installed Homebrew, you can install it by running the following command here or in the terminal:

In [ ]:
%%bash
sudo /bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"

### <p style="color:gold">1. Python</p> 
Once Homebrew is installed, you can install Python using the following command  
*check what you have before installing/upgrading*  
*you will need to quit and restart vsCode to use python once installed (or updated)*

In [4]:
%%bash
python3 --version
which python3

Python 3.12.2
/opt/homebrew/bin/python3


In [ ]:
%%bash
brew install python

### <p style="color:gold">2. boto3 and other Python requirements</p> 
*check what you have before installing/upgrading*  

In [5]:
%%bash
python3 -m pip show boto3

CalledProcessError: Command 'b'python3 -m pip show boto3\n'' returned non-zero exit status 1.

In [ ]:
pip install -U boto3

### <p style="color:gold">2.1 Chatbot requirements</p> 
*<p style="color:deeppink">If you are running a Jupyter in AWS Account, you ONLY need these 2 cells</p>*

In [ ]:
pip install -U langchain

In [ ]:
pip install langchain-community langchain-core

### <p style="color:gold">3. aws configure</p> 
*Configure aws configure with credentials, and a user that has all of the Bedrock IAM policies required*  
https://docs.aws.amazon.com/bedrock/latest/userguide/security_iam_id-based-policy-examples.html

In [ ]:
%%bash
aws sts get-caller-identity

### <p style="color:gold">4. Request Bedrock model access</p> 
*You must request access to the models required, you may need to provide use case details before you are able to request*  
*Make sure you request in the region you intend to use the models in, this lab is us-west-2*  
https://us-west-2.console.aws.amazon.com/bedrock/home?region=us-west-2#/modelaccess  

Models required in this lab:

* See code above for use of models and what access to request

#### Pricing
*Use 6 characters per token as an approximation for the number of tokens.*  
https://aws.amazon.com/bedrock/pricing/  
https://docs.aws.amazon.com/bedrock/latest/userguide/model-customization-prepare.html

<hr style="border:1px dotted;color:gold">